## Sentiment analyse twitter

In [9]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re

### We load an explore our labeled dataset 

labeld training set can be download from:http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

In [5]:
##This chunck is for reading in the train and test data, including a little bit of exploratory.

cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("C:\\Users\\Jeffr\\Downloads\\training.1600000.processed.noemoticon.csv",header=None, names=cols,encoding="ISO-8859–1")
# above line will be different depending on where you saved your data, and your file name
df.head()
df.drop(['id','date','query_string','user'],axis=1,inplace=True) ## Drop the rows that we dont need
df[df.sentiment == 4].head(10) ## Show some rows of sentiment 4 , looks like 4 is positive


,sentiment,text
800000,4,I LOVE @Health4UandPets u guys r the best!!
800001,4,im meeting up with one of my besties tonight! ...
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,4,Being sick can be really cheap when it hurts t...
800004,4,@LovesBrooklyn2 he has that effect on everyone
800005,4,@ProductOfFear You can tell him that I just bu...
800006,4,@r_keith_hill Thans for your response. Ihad al...
800007,4,"@KeepinUpWKris I am so jealous, hope you had a..."
800008,4,"@tommcfly ah, congrats mr fletcher for finally..."
800009,4,@e4VoIP I RESPONDED Stupid cat is helping me ...


In [3]:
df[df.sentiment == 0].head(10) ## Show some rows of sentiment 0, looks like 0 is negative

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [6]:
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1}) ## Map indicater 4 to 1
df.sentiment.value_counts() ## Check how much labeled is 800k sentiment 4 and 800k sentiment 0

1    800000
0    800000
Name: sentiment, dtype: int64

## We load our untagged dataset

In [119]:
%%time
# Verander in de regel hieronder de locatie van de data
tweets = pd.read_json("C:\\Users\\Jeffr\\Downloads\\mini_twitter_geotagged_clean.json", orient='records', lines=True)


Wall time: 3min 46s


In [121]:
# tweets.head()

tweets['ID'] = tweets.index ## Performance reasons and slicing for myself
tweets = tweets[['ID','CleanText','text']]

In [102]:
tweets.head()

,ID,CleanText,text
0,0,\n\n\nALL IN COLLUSION TOGETHER \n\n\n\n\n\n\n...,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...
1,1,he will do in one year all the things you shou...,@HillaryClinton he will do in one year all the...
2,2,clear deliberately throwing this racein 2007 h...,#CNN #newday clear #Trump deliberately throwin...
3,3,you wouldnt recognize a lie if it came from y...,"@realDonaldTrump, you wouldn't recognize a lie..."
4,4,Kid you know suing someone Thats the most beau...,"""Kid, you know, suing someone? Thats the most ..."


In [98]:
len(tweets)

46811

## Cleansing the tagged train data
The untagged dataset is already cleansed in the Preprocessing.ipynb
* Removal of http and www links
* Removal of hashtags #
* Removal of multiple spaces

In addition we delete enter and tab encoding. And words that are smaller than 3 letters.

In [12]:
def clean_string(row):
    # remove http and www links from string
    rem_hl = re.sub(r'(http\S+) | (www\S+) | (https\S+)', '', row['text'])
    
    # remove @.. from string
    rem_at_hl = re.sub(r'@(\w+) |@(\w+)', '', rem_hl)
    
    # remove hashtags #.. from string
    rem_hash_at_hl = re.sub(r'#(\w+) |@(\w+)', '', rem_at_hl)

    rem_inter = re.sub(r'[^\w\s]','', rem_hash_at_hl)
    
    # remove multiple spaces
    rem_space = re.sub(' +',' ',rem_inter)
    #print (re.search('[a-zA-Z]', rem_space))
    if (re.search('[a-zA-Z]', rem_space) == None):
        return("emptyStringRetured")
    else:
        return (rem_space)


In [15]:
df['text'] = df.apply(lambda row: clean_string(row), axis=1)

In [122]:
tweets['CleanText'] = tweets['CleanText'].replace('\n','')
tweets['CleanText'] = tweets['CleanText'].replace('\t','')

df['text'] = df['text'].replace('\n','')
df['text'] = df['text'].replace('\t','')

tweets['CleanText'] = tweets['CleanText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


tweets['CleanText'] =tweets['CleanText'].str.lower()
df['text'] = df['text'].str.lower()

C:\Users\Jeffr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jeffr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Jeffr\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [ ]:
np.sum(my_df.isnull().any(axis=1))
my_df.isnull().any(axis=0)

In [28]:
## This chuck is ment for further exploration
# df.isnull().any(axis=0)
# np.sum(df.isnull().any(axis=1))
# tweets.isnull().any(axis=0)
# np.sum(tweets.isnull().any(axis=1))
df.head() 
tweets.head()

,ID,CleanText
0,0,collusion together httpstco5gmnzq40v3
1,1,will year things should have done eight
2,2,clear deliberately throwing this racein 2007 k...
3,3,wouldnt recognize came from your mouth they co...
4,4,know suing someone thats most beautiful thing ...


## Word counts in labeld data

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()
cvec.fit(df.text)

neg_doc_matrix = cvec.transform(df[df.sentiment == 0].text)
pos_doc_matrix = cvec.transform(df[df.sentiment == 1].text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names()).transpose()

In [32]:
term_freq_df.columns = ['negative', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:10]

,negative,positive,total
have,82462,62039,144501
that,72583,68739,141322
just,63178,61874,125052
with,49919,64926,114845
this,52087,40890,92977
good,28551,60840,89391
like,40639,37106,77745
dont,44916,22009,66925
your,20752,44244,64996
today,36189,28400,64589


## Sentiment analyse --Step 1 splitting the data

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['sentiment'].values, test_size=0.2)

## Sentiment analyse -- Step 2 tokanize

In [47]:
import sklearn
vect = CountVectorizer() ## This is the tokanizer
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)


In [48]:
tf_train ##1280k

<1280000x396692 sparse matrix of type '<class 'numpy.int64'>'
	with 8556773 stored elements in Compressed Sparse Row format>

In [50]:
tf_test ##320k

<320000x396692 sparse matrix of type '<class 'numpy.int64'>'
	with 2069366 stored elements in Compressed Sparse Row format>

In [54]:
## This chunk is ment to check if the porportions are still right
sum(y_train) ## 640107   is the half of 1280 k in the train set
sum(y_test) ## 159893   is the half of 320k in the test set

159893

## Sentiment analyse -- Step 3 Building a classifier

In [79]:
# p = sum of all feature count vectors with label 1
p = tf_train[y_train==1].sum(0) + 1
# q = sum of all feature count vectors with label 0
q = tf_train[y_train==0].sum(0) + 1
# Notice that we add 1 to both count vectors to ensure that every token appear at least one time in each class.

# The log-count ratio r is:
r = np.log((p/p.sum()) / (q/q.sum()))
# And b:
b = np.log(len(p) / len(q))

# Predictions

# With calculated coefficients we can now generate predictions on test set. Since we are trying to fit a linear classifier, the form of the linear equation is:

# y = mx + b
pre_preds = tf_test @ r.T + b
preds = pre_preds.T > 0
accuracy = (preds == y_test).mean()
accuracy


0.75790625

The accuracy is 75.8 % meaning that the algorithm classisfy the correct label in 3 out of the 44 labels.

In [82]:
## Visualize the data
test_outcome = pd.DataFrame({'text':X_test, 'Positive':np.asarray(preds)[0]})
test_outcome.head(15)

,text,Outcome
0,bored motel again,False
1,here,False
2,hehe love that excuse blogging instead,True
3,misses girls,False
4,nobody wants lonely,False
5,waaaaaa wanna david archuletas summer tour,False
6,crazy hope tomorrow goes better mean didnt eve...,True
7,swans lost aboiut some homework,False
8,1030 already annoyed with kids,False
9,damn thunder lightning kept ughhh late work mood,False


## Sentiment analyse -- Step 4 Score out on our dataset

In [90]:
# Vectorize the unseen data
unseen = vect.transform(tweets.CleanText)
pre_unseen = unseen @ r.T + b
predictions_unseen = pre_unseen.T > 0

In [110]:
##Visualize the data
pd.options.display.max_rows  ## This is for showing all the text in the column (otherwise it will be half....)
pd.set_option('display.max_colwidth', -1)

Predictions_unseen_df = pd.DataFrame({'text':tweets.text, 'Positive':np.asarray(predictions_unseen)[0]})
Predictions_unseen_df.head(20)


,text,Positive
0,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,True
1,@HillaryClinton he will do in one year all the things you should have done in eight,True
2,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion",False
3,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm",False
4,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s",True
5,"@HillaryClinton you ARE the co-founder of ISIS, you crooked, evil, lying, witch. How can you live with yourself?",False
6,@Geraldanthro @NeilTurner_ @realDonaldTrump want to do a comparison try maimed Vets pre &amp; post Iraq pullout. Bar graph that. @washingtonpost,False
7,@mike4193496 @realDonaldTrump I TOTALLY CONCUR!! This Election is just CRA CRA n Corruption in our Gov is Mind Blowing!! Trump= Last Hope!!!,True
8,"@realDonaldTrump @elsolarverde What issues? Your idiot claim that she ""founded"" ISIS? Trump will go to Hell for lying &amp; stealing. Shame!",False
9,Can't stand @HillaryClinton or @realDonaldTrump?Take a look. They can win...don't settle. #15for15 #TeamGov #YouIn https://t.co/YK336aaH98,False


In [109]:
## This is a check if we have still all the data (46811) in the dataframe
len(Predictions_unseen_df) #46811
sum(Predictions_unseen_df.Positive) ## Amount of positive == 24217

24217

## Write back the dataset to share

In [120]:
# tweets = pd.read_json("C:\\Users\\Jeffr\\Downloads\\twitter_geotagged_clean.json", orient='records', lines=True)
# tweets["Positive"] = np.asarray(predictions_unseen)[0]
# tweets.head()
tweets2 = tweets
# tweets.to_json('C:\\Users\\Jeffr\\Downloads\\twitter_geotagged_clean_including_sentiment.json', orient='records', lines=True)